In [42]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pandas as pd
import matplotlib.pyplot as plt
import time
import os

In [43]:
# 求人ランキングページのURLのリスト
url = [
    'https://type.jp/rank/development/',
    'https://type.jp/rank/pm/',
    'https://type.jp/rank/infrastructure/',
    'https://type.jp/rank/engineer/',
    'https://type.jp/rank/sales/',
    'https://type.jp/rank/service/',
    'https://type.jp/rank/office/'
]

In [44]:
# CSVファイルの準備
csv_file_path = 'scraping.csv'
csv_columns = ['企業名', '順位', '年収', '設立', '従業員数', '資本金']

In [45]:
#リクエストを送信
def get_request(url):
    response = requests.get(url)
    return response

In [65]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os

def get_job_urls_from_ranking(ranking_url):
    """ランキングページから求人URLと順位を取得"""
    try:
        print(f"ランキングページにアクセス中: {ranking_url}")
        
        session = requests.Session()
        response = session.get(ranking_url, headers=headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 求人情報を含む要素を検索
        job_data = []
        job_items = soup.find_all('div', class_='mod-job-info-left')
        
        print(f"検出された求人アイテム数: {len(job_items)}")
        
        for rank, item in enumerate(job_items[:20], 1):
            try:
                # 求人リンクの取得
                link_elem = item.find('a', href=True)
                if link_elem:
                    link = link_elem.get('href')
                    full_url = 'https://type.jp' + link if not link.startswith('http') else link
                    
                    # 企業名の取得
                    company_name = item.find('p', class_='company').text.strip() if item.find('p', class_='company') else "企業名なし"
                    
                    # 給与情報の取得（右側の情報エリアから）
                    info_right = item.find_next('div', class_='mod-job-info-right')
                    if info_right:
                        salary_elem = info_right.find(['p', 'span'], class_=['data']) or \
                                    info_right.find(['p', 'span'], text=lambda x: x and '万円' in str(x))
                        salary = salary_elem.text.strip() if salary_elem else "給与情報なし"
                    else:
                        salary = "給与情報なし"
                    
                    job_data.append({
                        'url': full_url,
                        'rank': rank,
                        'company_name': company_name,
                        'salary': salary
                    })
                    
                    print(f"\n求人情報を取得: Rank {rank}")
                    print(f"企業名: {company_name}")
                    print(f"URL: {full_url}")
                    print(f"給与: {salary}")
            
            except Exception as e:
                print(f"個別求人の処理でエラー (Rank {rank}): {str(e)}")
        
        return job_data
            
    except Exception as e:
        print(f"ランキングページの処理でエラー: {str(e)}")
        return []

def get_company_info(job_url):
    """個別の求人ページから企業情報を取得"""
    try:
        print(f"求人ページにアクセス中: {job_url}")
        
        session = requests.Session()
        response = session.get(job_url, headers=headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 企業情報を含む要素を探す
        company_info = {
            'company_name': "情報なし",
            'established': "情報なし",
            'employees': "情報なし",
            'capital': "情報なし"
        }
        
        # 企業名の取得（複数のパターンを試行）
        company_elem = (
            soup.find('p', class_='corp-name') or
            soup.find('span', class_='company') or
            soup.find(['h1', 'h2'], class_=['company', 'corp-name'])
        )
        if company_elem:
            company_info['company_name'] = company_elem.text.strip()
        
        # 企業情報テーブルの検索
        info_table = soup.find('table', class_=['mod-table', 'company-info'])
        if info_table:
            rows = info_table.find_all('tr')
            for row in rows:
                header = row.find('th')
                value = row.find('td')
                if header and value:
                    header_text = header.text.strip()
                    if '設立' in header_text:
                        company_info['established'] = value.text.strip()
                    elif '従業員' in header_text:
                        company_info['employees'] = value.text.strip()
                    elif '資本金' in header_text:
                        company_info['capital'] = value.text.strip()
        
        return company_info
    
    except Exception as e:
        print(f"求人ページの処理でエラー: {str(e)}")
        return None
def main():
    all_data = []
    
    for ranking_url in url:
        print(f"\n{'='*50}")
        print(f"ランキングページの処理開始: {ranking_url}")
        time.sleep(3)  # アクセス間隔を設定
        
        # ランキングページから求人URLを取得
        job_data = get_job_urls_from_ranking(ranking_url)
        
        if job_data:
            print(f"\n{len(job_data)}件の求人情報を取得しました")
            # 各求人について企業情報を取得
            for job in job_data:
                time.sleep(2)
                company_info = get_company_info(job['url'])
                
                if company_info:
                    formatted_data = {
                        '企業名': company_info['company_name'],
                        '順位': job['rank'],
                        '年収': job['salary'],
                        '設立': company_info['established'],
                        '従業員数': company_info['employees'],
                        '資本金': company_info['capital']
                    }
                    all_data.append(formatted_data)
                    print(f"企業情報を取得: {company_info['company_name']}")
        else:
            print(f"{ranking_url} からデータを取得できませんでした")
    
    # 収集したデータをCSVに保存
    if all_data:
        write_to_csv(all_data)
        print(f"\n合計 {len(all_data)}件 のデータを収集しました")
    else:
        print("\nデータを収集できませんでした")

if __name__ == "__main__":
    main()


ランキングページの処理開始: https://type.jp/rank/
ランキングページにアクセス中: https://type.jp/rank/
検出された求人アイテム数: 20

求人情報を取得: Rank 1
企業名: 企業名なし
URL: https://type.jp/job-7/1342653_detail/
給与: 250～1000万円

求人情報を取得: Rank 2
企業名: 企業名なし
URL: https://type.jp/job-7/1343441_detail/
給与: 350～1000万円

求人情報を取得: Rank 3
企業名: 企業名なし
URL: https://type.jp/job-7/1333870_detail/
給与: 300～1000万円

求人情報を取得: Rank 4
企業名: 企業名なし
URL: https://type.jp/job-11/1343474_detail/
給与: 250～750万円

求人情報を取得: Rank 5
企業名: 企業名なし
URL: https://type.jp/job-7/1343814_detail/
給与: 300～700万円

求人情報を取得: Rank 6
企業名: 企業名なし
URL: https://type.jp/job-1/1023402_detail/
給与: 給与情報なし

求人情報を取得: Rank 7
企業名: 企業名なし
URL: https://type.jp/job-5/1028379_detail/
給与: 450～850万円

求人情報を取得: Rank 8
企業名: 企業名なし
URL: https://type.jp/job-5/1341471_detail/
給与: 給与情報なし

求人情報を取得: Rank 9
企業名: 企業名なし
URL: https://type.jp/job-1/1307769_detail/
給与: 250～1400万円

求人情報を取得: Rank 10
企業名: 企業名なし
URL: https://type.jp/job-7/1341482_detail/
給与: 350～600万円

求人情報を取得: Rank 11
企業名: 企業名なし
URL: https://type.jp/job-1/1

/var/folders/fn/tvnr48555n13vpphlc_rqvv00000gn/T/ipykernel_71998/2256525328.py:39: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  info_right.find(['p', 'span'], text=lambda x: x and '万円' in str(x))


求人ページにアクセス中: https://type.jp/job-7/1342653_detail/
企業情報を取得: 株式会社ＳＤキャリアサポート
求人ページにアクセス中: https://type.jp/job-7/1343441_detail/
企業情報を取得: 株式会社FEDELTA
求人ページにアクセス中: https://type.jp/job-7/1333870_detail/
企業情報を取得: 株式会社Ａｅｇｉｓ
求人ページにアクセス中: https://type.jp/job-11/1343474_detail/
企業情報を取得: ＧＭＯコネクト株式会社【東証プライム上場 GMOインターネットグループ】
求人ページにアクセス中: https://type.jp/job-7/1343814_detail/
企業情報を取得: 株式会社Ａｎｔｒａｃｅ
求人ページにアクセス中: https://type.jp/job-1/1023402_detail/
企業情報を取得: 日本マイクロソフト株式会社【ポジションマッチ登録】
求人ページにアクセス中: https://type.jp/job-5/1028379_detail/
企業情報を取得: サイボウズ株式会社
求人ページにアクセス中: https://type.jp/job-5/1341471_detail/
企業情報を取得: 亀田製菓株式会社【ポジションマッチ登録】
求人ページにアクセス中: https://type.jp/job-1/1307769_detail/
企業情報を取得: 株式会社Euphonauts
求人ページにアクセス中: https://type.jp/job-7/1341482_detail/
企業情報を取得: ゼロプライド株式会社
求人ページにアクセス中: https://type.jp/job-1/1318111_detail/
企業情報を取得: 株式会社トライ
求人ページにアクセス中: https://type.jp/job-7/1344012_detail/
企業情報を取得: 株式会社ＬＩＦＦＩＫ
求人ページにアクセス中: https://type.jp/job-7/1312221_detail/
企業情報を取得: 株式会社MIRDIS
求人ページにアクセス中: https:/

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os

# ユーザーエージェントの設定
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# スクレイピング対象のURL
url = [
    'https://type.jp/rank/',  # 総合
    'https://type.jp/rank/development/',  # 開発エンジニア
    'https://type.jp/rank/pm/',  # PM/PL/ITコンサル
    'https://type.jp/rank/infrastructure/',  # インフラ系エンジニア
    'https://type.jp/rank/engineer/',  # その他ITエンジニア
    'https://type.jp/rank/sales/',  # 営業
    'https://type.jp/rank/service/',  # サービス・販売
    'https://type.jp/rank/office/'  # オフィスワーク
]

def get_category_from_url(url):
    """URLから職種カテゴリーを判断"""
    category_mapping = {
        '/rank/': '総合',
        '/rank/development/': '開発エンジニア',
        '/rank/pm/': 'PM/PL/ITコンサル',
        '/rank/infrastructure/': 'インフラ系エンジニア',
        '/rank/engineer/': 'その他ITエンジニア',
        '/rank/sales/': '営業',
        '/rank/service/': '販売・サービス',
        '/rank/office/': '事務'
    }
    
    for path, category in category_mapping.items():
        if path in url:
            return category
    return '不明'

def get_job_urls_from_ranking(ranking_url):
    """ランキングページから求人URLと順位を取得"""
    try:
        print(f"ランキングページにアクセス中: {ranking_url}")
        
        session = requests.Session()
        response = session.get(ranking_url, headers=headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # カテゴリーを取得
        category = get_category_from_url(ranking_url)
        print(f"現在の職種カテゴリー: {category}")
        
        # 求人情報を含む要素を検索
        job_data = []
        job_items = soup.find_all('div', class_='mod-job-info-left')
        
        print(f"検出された求人アイテム数: {len(job_items)}")
        
        for rank, item in enumerate(job_items[:20], 1):
            try:
                # 求人リンクの取得
                link_elem = item.find('a', href=True)
                if link_elem:
                    link = link_elem.get('href')
                    full_url = 'https://type.jp' + link if not link.startswith('http') else link
                    
                    # 企業名の取得
                    company_name = item.find('p', class_='company').text.strip() if item.find('p', class_='company') else "企業名なし"
                    
                    # 職種の取得（求人タイトルから）
                    job_title = item.find('h2', class_='job-title')
                    job_title = job_title.text.strip() if job_title else "職種不明"
                    
                    # 給与情報の取得（右側の情報エリアから）
                    info_right = item.find_next('div', class_='mod-job-info-right')
                    if info_right:
                        salary_elem = info_right.find(['p', 'span'], class_=['data']) or \
                                    info_right.find(['p', 'span'], text=lambda x: x and '万円' in str(x))
                        salary = salary_elem.text.strip() if salary_elem else "給与情報なし"
                    else:
                        salary = "給与情報なし"
                    
                    job_data.append({
                        'url': full_url,
                        'rank': rank,
                        'category': category,
                        'job_title': job_title,
                        'company_name': company_name,
                        'salary': salary
                    })
                    
                    print(f"\n求人情報を取得: Rank {rank}")
                    print(f"カテゴリー: {category}")
                    print(f"職種: {job_title}")
                    print(f"企業名: {company_name}")
                    print(f"URL: {full_url}")
                    print(f"給与: {salary}")
            
            except Exception as e:
                print(f"個別求人の処理でエラー (Rank {rank}): {str(e)}")
        
        return job_data
            
    except Exception as e:
        print(f"ランキングページの処理でエラー: {str(e)}")
        return []

def get_company_info(job_url):
    """個別の求人ページから企業情報を取得"""
    try:
        print(f"求人ページにアクセス中: {job_url}")
        
        session = requests.Session()
        response = session.get(job_url, headers=headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 企業情報を含む要素を探す
        company_info = {
            'company_name': "情報なし",
            'established': "情報なし",
            'employees': "情報なし",
            'capital': "情報なし"
        }
        
        # 企業名の取得（複数のパターンを試行）
        company_elem = (
            soup.find('p', class_='corp-name') or
            soup.find('span', class_='company') or
            soup.find(['h1', 'h2'], class_=['company', 'corp-name'])
        )
        if company_elem:
            company_info['company_name'] = company_elem.text.strip()
        
        # 企業情報テーブルの検索
        info_table = soup.find('table', class_=['mod-table', 'company-info'])
        if info_table:
            rows = info_table.find_all('tr')
            for row in rows:
                header = row.find('th')
                value = row.find('td')
                if header and value:
                    header_text = header.text.strip()
                    if '設立' in header_text:
                        company_info['established'] = value.text.strip()
                    elif '従業員' in header_text:
                        company_info['employees'] = value.text.strip()
                    elif '資本金' in header_text:
                        company_info['capital'] = value.text.strip()
        
        return company_info
    
    except Exception as e:
        print(f"求人ページの処理でエラー: {str(e)}")
        return None

def write_to_csv(data):
    """データをCSVファイルに書き出す"""
    try:
        filename = 'job_ranking_data.csv'
        file_exists = os.path.isfile(filename)
        
        with open(filename, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=['企業名', '順位', 'カテゴリー', '職種', '年収', '設立', '従業員数', '資本金'])
            
            if not file_exists:
                writer.writeheader()
            
            for item in data:
                formatted_data = {
                    '企業名': item['company_name'],
                    '順位': item['rank'],
                    'カテゴリー': item['category'],
                    '職種': item['job_title'],
                    '年収': item['salary'],
                    '設立': item.get('established', '情報なし'),
                    '従業員数': item.get('employees', '情報なし'),
                    '資本金': item.get('capital', '情報なし')
                }
                writer.writerow(formatted_data)
            
        print(f"\nデータを {filename} に保存しました")
    
    except Exception as e:
        print(f"CSV書き込み時にエラー: {str(e)}")

def main():
    all_data = []
    
    for ranking_url in url:
        print(f"\n{'='*50}")
        print(f"ランキングページの処理開始: {ranking_url}")
        time.sleep(3)  # アクセス間隔を設定
        
        # ランキングページから求人URLを取得
        job_data = get_job_urls_from_ranking(ranking_url)
        
        if job_data:
            print(f"\n{len(job_data)}件の求人情報を取得しました")
            # 各求人について企業情報を取得
            for job in job_data:
                time.sleep(2)
                company_info = get_company_info(job['url'])
                
                if company_info:
                    # 企業情報をjobデータに統合
                    job.update(company_info)
                    all_data.append(job)
                    print(f"企業情報を取得: {company_info['company_name']}")
        else:
            print(f"{ranking_url} からデータを取得できませんでした")
    
    # 収集したデータをCSVに保存
    if all_data:
        write_to_csv(all_data)
        print(f"\n合計 {len(all_data)}件 のデータを収集しました")
    else:
        print("\nデータを収集できませんでした")

if __name__ == "__main__":
    main()


ランキングページの処理開始: https://type.jp/rank/
ランキングページにアクセス中: https://type.jp/rank/
現在の職種カテゴリー: 総合
検出された求人アイテム数: 20

求人情報を取得: Rank 1
カテゴリー: 総合
職種: 職種不明
企業名: 企業名なし
URL: https://type.jp/job-7/1343441_detail/
給与: 350～1000万円

求人情報を取得: Rank 2
カテゴリー: 総合
職種: 職種不明
企業名: 企業名なし
URL: https://type.jp/job-7/1342653_detail/
給与: 250～1000万円

求人情報を取得: Rank 3
カテゴリー: 総合
職種: 職種不明
企業名: 企業名なし
URL: https://type.jp/job-7/1333870_detail/
給与: 300～1000万円

求人情報を取得: Rank 4
カテゴリー: 総合
職種: 職種不明
企業名: 企業名なし
URL: https://type.jp/job-11/1343474_detail/
給与: 250～750万円

求人情報を取得: Rank 5
カテゴリー: 総合
職種: 職種不明
企業名: 企業名なし
URL: https://type.jp/job-7/1343814_detail/
給与: 300～700万円

求人情報を取得: Rank 6
カテゴリー: 総合
職種: 職種不明
企業名: 企業名なし
URL: https://type.jp/job-1/1023402_detail/
給与: 給与情報なし

求人情報を取得: Rank 7
カテゴリー: 総合
職種: 職種不明
企業名: 企業名なし
URL: https://type.jp/job-5/1028379_detail/
給与: 450～850万円

求人情報を取得: Rank 8
カテゴリー: 総合
職種: 職種不明
企業名: 企業名なし
URL: https://type.jp/job-5/1341471_detail/
給与: 給与情報なし

求人情報を取得: Rank 9
カテゴリー: 総合
職種: 職種不明
企業名: 企業名なし
URL: https://ty

/var/folders/fn/tvnr48555n13vpphlc_rqvv00000gn/T/ipykernel_71998/2926069580.py:82: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  info_right.find(['p', 'span'], text=lambda x: x and '万円' in str(x))


求人ページにアクセス中: https://type.jp/job-7/1343441_detail/
企業情報を取得: 株式会社FEDELTA
求人ページにアクセス中: https://type.jp/job-7/1342653_detail/
企業情報を取得: 株式会社ＳＤキャリアサポート


KeyboardInterrupt: 